In [2]:
import pyterrier as pt
if not pt.started():
  print ("not")  
  pt.init()

not
PyTerrier 0.5.0 has loaded Terrier 5.4 (built by craigm on 2021-01-16 14:17)


In [41]:
#!pip install --upgrade fastrank lightgbm 
#!pip install python-terrier

In [42]:
import pickle
from my_functions import extractorRoberta
import torch
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
my_extractor = extractorRoberta(my_device = device, model_path = '/notebook/cqas/external_pretrained_models/')
print ("loaded extractors")

/notebook/cqas/external_pretrained_models/roberta.hdf5
/notebook/cqas/external_pretrained_models/vocab_dir
encoder loaded
indexer loaded
model path  /notebook/cqas/external_pretrained_models/roberta.hdf5
model loaded
reader loaded
loaded extractors


In [3]:
def extract_objs_asp(model_for_extraction, input_string):
    model_for_extraction.from_string(input_string)
    obj1, obj2, predicates, aspects = model_for_extraction.get_params()
    return (obj1.lower(), obj2.lower(), predicates, aspects)


def count_score1(text, nlu_tuple):
    (obj1, obj2, pred, asp) = nlu_tuple
    r = 1.0
    if (len(obj1) != 0 and len(obj2) != 0):
        if (len(pred) != 0):
            pred = re.sub('[!#?,.:";]', '', pred[0])
            if (obj1 in text and obj2 in text and pred in text):
                r += 1.0
        if (len(asp) != 0):
            asp = re.sub('[!#?,.:";]', '', asp[0])
            if (obj1 in text and obj2 in text and asp in text):
                r += 1.0
        elif (obj1 in text and obj2 in text):
            r = 1.5
        elif (obj1 in text or obj2 in text):
            r = 1.2
    else:
        if (obj1) in text or (obj2) in text:
            r = 1.2
    return r

def count_score_nlu(nlu_tuple):
    if (len(nlu_tuple[0]) == 0):
        return 0.0
    else: return 1.0


def count_score(text, nlu_tuple):
    (obj1, obj2, preds, asps) = nlu_tuple
    r = 0.0
    text = cleanhtml(text)
    if (len(obj1) != 0 and len(obj2) != 0):
        if (obj1 in text):
            r += 1.0
        if (obj2 in text):
            r += 1.0
        for asp in asps:
            if asp in text:
                r += 1.5
        for pred in preds:
            if pred in text:
                r += 1.0
    else:
        if ((obj1) in text and len(obj1)!= 0) or (obj2 in text and len(obj2) != 0):
            r = 1.0
    return r

def count_score_obj(text, nlu_tuple):
    (obj1, obj2, preds, asps) = nlu_tuple
    r = 0.0
    text = cleanhtml(text)
    if (len(obj1) != 0 and obj1 in text):
        r += 1.0
    if (len(obj2) != 0 and obj2 in text):
        r += 1.0
    return r

def count_score_asp_pred(text, nlu_tuple):
    (obj1, obj2, preds, asps) = nlu_tuple
    r = 0.0
    text = cleanhtml(text)
    o1 = (len(obj1) != 0 and obj1 in text)
    o2 = (len(obj2) != 0 and obj2 in text)
    if (o1 or o2):
        for asp in asps:
            if asp in text:
                r += 0.5
        for pred in preds:
            if pred in text:
                r += 0.5
    return r


def make_scores_obj(query, answers):
    print ("make_scores_obj")
    (obj1, obj2, pred, asp) = extract_objs_asp(extr, query)
    print ("in make scores", obj1, obj2, pred, asp)
    scores_answers = [count_score(cleanhtml(answer), (obj1, obj2, pred, asp)) for answer in answers]
    return scores_answers

In [6]:
from xml.dom import minidom

def read_xml(filename):
    # convert file filename to list of tuples (number_of_topic, title_of_topic) 
    # input: filename string
    # output: list of corresponding tuples
    answer_list = []
    xmldoc = minidom.parse(filename)
    itemlist = xmldoc.getElementsByTagName('topics')
    print(len(itemlist))
    print(itemlist)
    topic_list = itemlist[0].getElementsByTagName('topic')
    print (len(topic_list))
    for topic in topic_list:
        tuple_for_add = tuple((topic.getElementsByTagName('number')[0].firstChild.nodeValue, topic.getElementsByTagName('title')[0].firstChild.nodeValue))
        answer_list.append(tuple_for_add)
    return answer_list

topics_2020 = read_xml('topics-task-2.xml')

1
[<DOM Element: topics at 0x7fa9d6071eb0>]
50


In [7]:
topics_2020[30:45]

[('31', '\nWhich has more caffeine, coffee or tea?\n'),
 ('32', '\nWhich is better, LED or LCD Reception Displays?\n'),
 ('33', '\nWhat is better: ASP or PHP?\n'),
 ('34',
  '\nWhat is better for the environment, a real or a fake Christmas tree?\n'),
 ('35', '\nDo you prefer tampons or pads?\n'),
 ('36', '\nWhat IDE is better for Java: NetBeans or Eclipse?\n'),
 ('37',
  '\nIs OpenGL better than Direct3D in terms of portability to different platforms?\n'),
 ('38',
  '\nWhat are the differences between MySQL and PostgreSQL in performance?\n'),
 ('39', '\nIs Java code more readable than code written in Scala?\n'),
 ('40',
  '\nWhich operating system has better performance: Windows 7 or Windows 8?\n'),
 ('41', '\nWhich smartphone has a better battery life: Xperia or iPhone?\n'),
 ('42', '\nWhich four wheel truck is better: Ford or Toyota?\n'),
 ('43',
  '\nShould I prefer a Leica camera over Nikon for portrait photographs?\n'),
 ('44', '\nWhich company has a larger capitalization: Apple o

In [46]:
import requests
#r = requests.post('http://10.30.99.211:8261/gpt_small', data = "What is better for deep learning Python or Matlab?")
#print (r.status_code)
import requests
from xml.dom import minidom
import sys
import argparse

#numpy
import numpy as np

Read retireved documents and qrels:

In [8]:
import pickle


with open('/notebook/touche/list_of_un_answ.pkl', 'rb') as f:
    answers_2020 = pickle.load(f)
    
with open('/notebook/touche2021/touche2020-task2-relevance-withbaseline.qrels', 'r') as f:
    qrels_lines = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
qrels = [x.strip().split() for x in qrels_lines] 


Create qrels dict:

In [9]:
qrels_dict = {}
for elem in qrels:
    query, noninf, docno, rank = elem
    if (query in qrels_dict.keys()):
        qrels_dict[query].append((docno, rank))
    else:
        qrels_dict[query] = []
        qrels_dict[query].append((docno, rank))

In [16]:
qrels_dict['34'][:20]

[('clueweb12-0815wb-21-00423', '2'),
 ('clueweb12-0108wb-27-03924', '2'),
 ('clueweb12-0001wb-84-27874', '2'),
 ('clueweb12-0004wb-25-35255', '2'),
 ('clueweb12-0007wb-69-30265', '2'),
 ('clueweb12-0103wb-12-24884', '1'),
 ('clueweb12-0810wb-94-06721', '2'),
 ('clueweb12-0101wb-61-12535', '1'),
 ('clueweb12-0311wb-03-10308', '2'),
 ('clueweb12-1010wb-21-13067', '0'),
 ('clueweb12-0010wb-73-33411', '2'),
 ('clueweb12-1313wb-65-05083', '2'),
 ('clueweb12-1411wb-38-10889', '1'),
 ('clueweb12-1414wb-05-16226', '2'),
 ('clueweb12-1010wb-84-02332', '0'),
 ('clueweb12-1516wb-43-24623', '1'),
 ('clueweb12-1406wb-06-02996', '1'),
 ('clueweb12-0109wb-54-20272', '0'),
 ('clueweb12-0309wb-64-08181', '2'),
 ('clueweb12-0310wb-87-04609', '1')]

In [ ]:
qrels_dict['34'][:10]

In [24]:
a = [elem for elem in answers_2020['34'] if elem[1] == 'clueweb12-1010wb-84-02332']
a

[(304.44406,
  'clueweb12-1010wb-84-02332',
  'voice in the dark - part 4',
  'friday tips volume twenty five » focus organic.com – this entry is part 25 of 103 in the series eco-friendly friday november 28th’s tip christmas trees: stuck between choosing a real christmas tree or a fake one?')]

In [18]:
a = [elem for elem in answers_2020['34'] if elem[1] == 'clueweb12-1010wb-84-02332']
a # 0

[(304.44406,
  'clueweb12-1010wb-84-02332',
  'voice in the dark - part 4',
  'friday tips volume twenty five » focus organic.com – this entry is part 25 of 103 in the series eco-friendly friday november 28th’s tip christmas trees: stuck between choosing a real christmas tree or a fake one?')]

In [23]:
a = [elem for elem in answers_2020['34'] if elem[1] == 'clueweb12-0001wb-84-27874']
a # 2

[(1652.0702,
  'clueweb12-0001wb-84-27874',
  'christmas tree allergy - avoid pine, fir, spruce, or artificial christmas',
  'how to avoid allergies during christmas tree season. about.com health&#x27;s disease and condition content is reviewed by our medical review board real or artificial? there is so much confusing information out there about which is better for your health and the environment.')]

In [21]:
a = [elem for elem in answers_2020['34'] if elem[1] == 'clueweb12-0310wb-87-04609']
a # 1

[(2384.4077,
  'clueweb12-0310wb-87-04609',
  'is a fake christmas tree the &#x27;green&#x27; choice? - msn money',
  'you may think you&#x27;re saving a tree, but the plastic alternative has problems too. while the debate rages on, we&#x27;ve got some better ways to help the planet this christmas. which is &quot;greener,&quot; an artificial christmas tree or a real one?')]

In [51]:
def clean_punct(s):
    s = re.sub(r'[^\w\s]','',s)
    return s

def cleanhtml(raw_html):
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', raw_html)
    return cleantext.lower()


In [20]:
qrels[:3]

[['1', '0', 'clueweb12-0001wb-05-12311', '0'],
 ['1', '0', 'clueweb12-1811wb-62-08424', '1'],
 ['1', '0', 'clueweb12-1811wb-62-08423', '1']]

In [21]:
for elem in qrels:
    if (elem[2] == 'clueweb12-1214wb-88-29751'):
        print (elem)

['1', '0', 'clueweb12-1214wb-88-29751', '2']


In [22]:
qrels_df_test

NameError: name 'qrels_df_test' is not defined

Create proper pandas df documents and qrels

In [ ]:
import pandas as pd
import re

info_df = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"],dtype=object)
info_df_test = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"], dtype=object)
qrels_df = pd.DataFrame(columns=["qid", "docno", "label"],dtype=object)
qrels_df_test = pd.DataFrame(columns=["qid", "docno", "label"],dtype=object)

for elem in topics_2020:
        qid, query = elem[0], elem[1].strip('\n')
        query = re.sub(r'[^\w\s]','',query)
        query = cleanhtml(query)
        my_extractor.from_string(query)
        structures = my_extractor.get_params()

        for ind, answer in enumerate(answers_2020[qid]):
            docno = answer[1]
            score = answer[0]
            text = answer[3]
            label = [el[3] for el in qrels if el[2]==docno]
            nlu_score = count_score(text, structures)
            objs_score = count_score_obj(text, structures)
            ap_score = count_score_asp_pred(text, structures)
            is_retrieved = count_score_nlu(structures)
            if (True):
                df_row = {"qid":qid, "query":query, "docno":docno, "text":text, "baseline_scores":score, "is_retrieved":is_retrieved, "ap_score":ap_score, "objs_score":objs_score}
                if int(qid) < 40:          
                    info_df = info_df.append(df_row, ignore_index= True)
                else:
                    info_df_test = info_df_test.append(df_row, ignore_index= True)
                
                for qrel in qrels_dict[qid]:
                    docno, label = qrel
                    df_row = {"qid":qid, "docno":docno, "label":label}
                if (int(qid) < 40):
                    qrels_df = qrels_df.append(df_row, ignore_index= True)
                else:
                    qrels_df_test = qrels_df_test.append(df_row, ignore_index= True)



#textscorerTf = pt.text.scorer(body_attr="text", wmodel="Tf")
#rtr = textscorerTf.transform(

In [53]:
print(len(info_df),len(qrels_df))

36798 36798


In [54]:
info_df.head()

,qid,query,docno,text,baseline_scores,is_retrieved,ap_score,objs_score
0,1,what is the difference between sex and love,clueweb12-1214wb-88-29751,sex may or may not include penetration. differ...,2406.7341,0.0,0.0,0.0
1,1,what is the difference between sex and love,clueweb12-1811wb-62-08418,having ’sex’ and a ‘rape’ are two completely d...,2396.6697,0.0,0.0,0.0
2,1,what is the difference between sex and love,clueweb12-0200wb-79-18105,"home &gt; articles &gt; sex, sexuality &amp; p...",2270.9827,0.0,0.0,0.0
3,1,what is the difference between sex and love,clueweb12-1311wb-38-04762,home &gt;&gt;&gt; sex education 2.0 &gt;&gt;&g...,2096.9185,0.0,0.0,0.0
4,1,what is the difference between sex and love,clueweb12-0200tw-85-01106,things have changed so much and it has been ye...,2010.6464,0.0,0.0,0.0


### Simple ranking

In [55]:
from pyterrier import text

In [56]:
from pyterrier.text import scorer

In [57]:
def create_featured_dataset(some_df):
    from pyterrier import text
    from pyterrier.text import scorer
    print (0)
    textscorerTf = text.scorer(body_attr="text", wmodel='BM25', sort=False)
    rtr_bm = textscorerTf.transform(some_df)
    print (1)
    textscorerTf = text.scorer(body_attr="text", wmodel='Tf')
    rtr_tf = textscorerTf.transform(some_df)
    print (2)
    textscorerTf = text.scorer(body_attr="text", wmodel='PL2')
    rtr_pl2 = textscorerTf.transform(some_df)
    print (3)
    textscorerTf = text.scorer(body_attr="text", wmodel='DFIC')
    rtr_dfic = textscorerTf.transform(some_df)
    print (4)
    rtr_pl2_for_merge = rtr_pl2[['qid', 'docno', 'score']]
    rtr_pl2_for_merge = rtr_pl2_for_merge.rename(columns={"score": "score_pl2"})
    print (5)
    rtr_tf_for_merge = rtr_tf[['qid', 'docno', 'score']]
    rtr_tf_for_merge = rtr_tf_for_merge.rename(columns={"score": "score_tf"})
    print (6)
    rtr_bm_for_merge = rtr_bm[['qid', 'docno', 'score']]
    rtr_bm_for_merge = rtr_bm_for_merge.rename(columns={"score": "score_bm"})
    print (7)
    rtr_dfic_for_merge = rtr_dfic[['qid', 'docno', 'score']]
    rtr_dfic_for_merge = rtr_dfic_for_merge.rename(columns={"score": "score_dfic"})
    print (8)
    result = pd.merge(rtr_pl2_for_merge, rtr_tf_for_merge, on=["qid", "docno"])
    result = pd.merge(result, rtr_bm_for_merge, on=["qid", "docno"])
    result = pd.merge(result, rtr_dfic_for_merge, on=["qid", "docno"])
    result = pd.merge(result, some_df, on=["qid", "docno"])
    zipped = [result["score_pl2"], result["score_tf"], result["score_bm"], result["score_dfic"], result['baseline_scores'], result["is_retrieved"], result["ap_score"], result["objs_score"]]
    unzipped_object = zip(*zipped)
    unzipped_list = list(unzipped_object)
    list_of_features = [np.array(elem) for elem in unzipped_list]
    result['features'] = list_of_features
    return result



In [27]:
type(info_df['qid'][9])

str

In [58]:
result = create_featured_dataset(info_df)

0
1
2
3
4
5
6
7
8


In [59]:
test_ds = create_featured_dataset(info_df_test)

0
1
2
3
4
5
6
7
8


In [62]:
qrels_df_test.head()

,qid,docno,label
0,40,clueweb12-0500tw-35-13169,1
1,40,clueweb12-0500tw-35-13169,1
2,40,clueweb12-0500tw-35-13169,1
3,40,clueweb12-0500tw-35-13169,1
4,40,clueweb12-0500tw-35-13169,1


In [63]:
test_ds.to_pickle("test_ds.pkl")
qrels_df_test.to_pickle("qrels_df_test.pkl")

In [172]:
set(list(qrels_df_test['label']))

{'0', '1', '2'}

In [42]:
def write_qrels(output_dir, name, rtr):
    print ("rtr in write qrels ", rtr.head())
    qids = rtr['qid']
    Q0s = [0 for elem in qids]
    docs = rtr['docno']
    ranks = rtr['rank']
    score = rtr['score']
    tags = [name for elem in qids]
    common_list = list(zip(qids, Q0s, docs, ranks, score, tags))
    print ("common_list ", common_list[:3])
    with open(output_dir + name + '.qrels', 'w') as fp:
        fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))
    print ("written " + name +'.txt')

In [43]:
write_qrels("/notebook/touche2021/output", "baseline_test", qrels_df_test)

rtr in write qrels    qid                      docno rank             score
0  40  clueweb12-1100tw-55-05937    1  75.8093116910726
1  40  clueweb12-1700wb-29-15033    2  69.9706035082827
2  40  clueweb12-1700tw-30-03488    3  65.9923622033137
3  40  clueweb12-1300tw-27-17207    4  51.7581117464051
4  40  clueweb12-0500tw-35-13169    5   51.276620362775
common_list  [('40', 0, 'clueweb12-1100tw-55-05937', '1', '75.8093116910726', 'baseline_test'), ('40', 0, 'clueweb12-1700wb-29-15033', '2', '69.9706035082827', 'baseline_test'), ('40', 0, 'clueweb12-1700tw-30-03488', '3', '65.9923622033137', 'baseline_test')]
written baseline_test.txt


In [156]:
!pwd

/notebook/touche2021


In [75]:
assert set(test_ds['qid']) == set(qrels_df_test['qid'])

## Write B.beggins qrels

In [ ]:
trec_eval/results/official2020_tracks.qrels/official2020_tracks.qrels

In [3]:
!pwd

/notebook/touche2021


In [ ]:
touche2021/runs-task-2/Bilbo Baggins/Run/run.txt

In [60]:
with open("/notebook/touche2021/runs-task-2/Bilbo Baggins/Run/run.txt") as f:
    content = f.readlines()
# you may also want to remove whitespace characters like `\n` at the end of each line
content = [x.strip().split() for x in content]

In [61]:
content[20]

['1',
 'Q0',
 'clueweb12-0004wb-91-15141',
 '21',
 '17.2093665154127',
 'ul-t2-voelkerschlacht']

In [65]:
import pandas as pd

#info_df = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"],dtype=object)
#info_df_test = pd.DataFrame(columns=["qid", "query", "docno", "text", "baseline_scores", "is_retrieved", "ap_score", "objs_score"], dtype=object)
qrels_df = pd.DataFrame(columns=["qid", "docno", "rank", "score"],dtype=object)
qrels_df_test = pd.DataFrame(columns=["qid", "docno", "rank", "score"],dtype=object)

for ind, answer in enumerate(content):
    if (ind%1000 == 0):
        print (ind)
    docno = answer[2]
    qid = answer[0]
    rank = answer[3]
    score = answer[4]
    df_row = {"qid":qid, "docno":docno, "rank":rank, "score":score}
    qrels_df_test = qrels_df_test.append(df_row, ignore_index= True)

0
1000
2000
3000
4000


In [66]:
def write_qrels(output_dir, name, rtr):
    print ("rtr in write qrels ", rtr.head())
    qids = rtr['qid']
    Q0s = [0 for elem in qids]
    docs = rtr['docno']
    ranks = rtr['rank']
    score = rtr['score']
    tags = [name for elem in qids]
    common_list = list(zip(qids, Q0s, docs, ranks, score, tags))
    print ("common_list ", common_list[:3])
    with open(output_dir + name + '.qrels', 'w') as fp:
        fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))
    print ("written " + name +'.txt')

In [67]:
write_qrels("/notebook/touche2021/output/", "big_top_1", qrels_df_test)

rtr in write qrels    qid                      docno rank             score
0   1  clueweb12-0010wb-73-33601    1  85.5517282896202
1   1  clueweb12-1400tw-53-02910    2  73.9870753533032
2   1  clueweb12-0307wb-05-31620    3  39.9813264103379
3   1  clueweb12-0201wb-90-06213    4  37.1568135522378
4   1  clueweb12-0506wb-26-02303    5  35.7838688603153
common_list  [('1', 0, 'clueweb12-0010wb-73-33601', '1', '85.5517282896202', 'big_top_1'), ('1', 0, 'clueweb12-1400tw-53-02910', '2', '73.9870753533032', 'big_top_1'), ('1', 0, 'clueweb12-0307wb-05-31620', '3', '39.9813264103379', 'big_top_1')]
written big_top_1.txt


In [ ]:
trec_eval -q -c -m ndcg 

In [41]:
qrels_df_test.head(15)

,qid,docno,rank,score
0,40,clueweb12-1100tw-55-05937,1,75.8093116910726
1,40,clueweb12-1700wb-29-15033,2,69.9706035082827
2,40,clueweb12-1700tw-30-03488,3,65.9923622033137
3,40,clueweb12-1300tw-27-17207,4,51.7581117464051
4,40,clueweb12-0500tw-35-13169,5,51.276620362775
5,40,clueweb12-0700tw-80-23495,6,45.597702384667
6,40,clueweb12-1705wb-53-33166,7,42.9273937942786
7,40,clueweb12-1515wb-18-13784,8,41.6086756489246
8,40,clueweb12-1913wb-50-22864,9,38.5421679948438
9,40,clueweb12-1612wb-21-28532,10,37.2138925975698


## Ranking with LTR

In [154]:
#bm25 = pt.BatchRetrieve(index, wmodel="BM25")
#tf = pt.BatchRetrieve(index, wmodel="Tf")
#pl2 = pt.BatchRetrieve(index, wmodel="PL2")
#pipeline = (rtr_tf ** rtr_pl2)

In [34]:
def add_ranks1(rtr : pd.DataFrame) -> pd.DataFrame:
    """
        Canonical method for adding a rank column which is calculated based on the score attribute
        for each query. Note that the dataframe is NOT sorted by this operation.
        Arguments
            df: dataframe to create rank attribute for
    """
    rtr.drop(columns=["rank"], errors="ignore", inplace=True)
    if len(rtr) == 0:
        rtr["rank"] = pd.Series(index=rtr.index, dtype='int64')
        return rtr
    print (0)
    # -1 assures that first rank will be FIRST_RANK
    rtr["rank"] = rtr.groupby("qid", sort=False)["score"].rank(ascending=False, method="first").astype(int) -1 + 1
    print (1)
    if True:
        rtr.sort_values(["qid", "rank"], ascending=[True,True], inplace=True)
    return rtr

def transform(model, test_DF):
    """
    Predicts the scores for the given topics.

    Args:
        topicsTest(DataFrame): A dataframe with the test topics.
    """
    test_DF = test_DF.copy()

    # check for change in number of features
    found_numf = test_DF.iloc[0].features.shape[0]
    if model.num_f is not None:
        if found_numf != model.num_f:
            raise ValueError("Expected %d features, but found %d features" % (model.num_f, found_numf))
    if hasattr(model.learner, 'feature_importances_'):
        if len(model.learner.feature_importances_) != found_numf:
            raise ValueError("Expected %d features, but found %d features" % (len(model.learner.feature_importances_), found_numf))

    test_DF["score"] = model.learner.predict(np.stack(test_DF["features"].values))
    test_DF["qid"] = [int(elem) for elem in test_DF["qid"]]
    return add_ranks1(test_DF)

In [35]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=10)
rf_pipe = pt.ltr.apply_learned_model(rf)
rf_pipe.fit(result, qrels_df)
answs = transform(rf_pipe, test_ds)
#pt.Experiment([result, rf_pipe], result, qrels_df, ["map"], names=["BM25 Baseline", "LTR"])

0
1


In [12]:
def write_qrels(name, rtr):
    qids = rtr['qid']
    Q0s = [0 for elem in qids]
    docs = rtr['docno']
    #scores = rtr['label']
    ranks = rtr['rank']
    tags = [name for elem in qids]
    common_list = list(zip(qids, Q0s, docs, ranks))
    with open(name +'.qrels', 'w') as fp:
        fp.write('\n'.join('%s %s %s %s %s %s' % x for x in common_list))
    print ("written " + name +'_test.qrels')

## LightGMB

In [37]:
import lightgbm as lgb

# this configures LightGBM as LambdaMART
lmart_l = lgb.LGBMRanker(
    task="train",
    silent=False,
    min_data_in_leaf=1,
    min_sum_hessian_in_leaf=1,
    max_bin=255,
    num_leaves=31,
    objective="lambdarank",
    metric="ndcg",
    ndcg_eval_at=[10],
    ndcg_at=[10],
    eval_at=[10],
    learning_rate= .1,
    importance_type="gain",
    num_iterations=100,
    early_stopping_rounds=5
)

rf_pipe = pt.ltr.apply_learned_model(lmart_l, form="ltr")

#rf_pipe.learner.fit(np.stack(train_DF["features"].values), train_DF["label"].values)
rf_pipe.fit(result, qrels_df, test_ds, qrels_df_test)
#answs = transform(rf_pipe, test_ds)



[LightGBM] [Warning] early_stopping_round is set=5, early_stopping_rounds=5 will be ignored. Current value: early_stopping_round=5
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] min_sum_hessian_in_leaf is set=1, min_child_weight=0.001 will be ignored. Current value: min_sum_hessian_in_leaf=1
[LightGBM] [Warning] min_data_in_leaf is set=1, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=1
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000621 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1055
[LightGBM] [Info] Number of data points in the train set: 45473, number of used features: 8
[L

/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/lightgbm/engine.py:148: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))
/opt/.pyenv/versions/3.7.4/lib/python3.7/site-packages/lightgbm/engine.py:153: UserWarning: Found `early_stopping_rounds` in params. Will use it instead of argument
  _log_warning("Found `{}` in params. Will use it instead of argument".format(alias))


In [39]:
rf_lgbm = pt.ltr.apply_learned_model(lmart_l, form="ltr")
rf_lgbm.fit(train_topics, train_qrels, topics_and_results_Valid = test_topics, qrelsValid = test_qrels)
answs = transform(rf_lgbm, test_topics)


NameError: name 'train_topics' is not defined

In [116]:
result.head()

,qid,docno,score_pl2,score_tf,score_bm,score_dfic,query,text,baseline_scores,is_retrieved,ap_score,objs_score,label,features
0,1,clueweb12-0200wb-79-18105,11.626052,9.0,23.103274,21.680909,what is the difference between sex and love,"home &gt; articles &gt; sex, sexuality &amp; p...",2270.98270,0.0,0.0,0.0,2,"[11.62605208595209, 9.0, 23.103273698299724, 2..."
1,1,clueweb12-1214wb-88-29751,11.339902,10.0,22.708421,21.520754,what is the difference between sex and love,sex may or may not include penetration. differ...,2406.73410,0.0,0.0,0.0,2,"[11.339901919789828, 10.0, 22.708421284786994,..."
2,1,clueweb12-0001wb-05-12311,8.889443,5.0,19.074886,17.001041,what is the difference between sex and love,her mantra &quot;no more bad dates&quot; is a ...,872.68384,0.0,0.0,0.0,0,"[8.889443285070843, 5.0, 19.074885717934308, 1..."
3,1,clueweb12-1400tw-53-02910,7.796204,4.0,17.229402,15.395087,what is the difference between sex and love,"leave a comment in the past, i’ve had sex, i’v...",1970.16040,0.0,0.0,0.0,2,"[7.796203967373771, 4.0, 17.22940189701382, 15..."
4,1,clueweb12-1801wb-11-09209,7.637733,4.0,14.292758,13.839917,what is the difference between sex and love,nor can we make love without god for he is the...,1178.09250,0.0,0.0,0.0,2,"[7.637732581490534, 4.0, 14.292757865862576, 1..."


In [115]:
train_DF.head()

,qid,docno,score_pl2,score_tf,score_bm,score_dfic,query,text,baseline_scores,is_retrieved,ap_score,objs_score,label_x,features,label_y
0,1,clueweb12-0200wb-79-18105,11.626052,9.0,23.103274,21.680909,what is the difference between sex and love,"home &gt; articles &gt; sex, sexuality &amp; p...",2270.98270,0.0,0.0,0.0,2,"[11.62605208595209, 9.0, 23.103273698299724, 2...",0
1,1,clueweb12-1214wb-88-29751,11.339902,10.0,22.708421,21.520754,what is the difference between sex and love,sex may or may not include penetration. differ...,2406.73410,0.0,0.0,0.0,2,"[11.339901919789828, 10.0, 22.708421284786994,...",0
2,1,clueweb12-0001wb-05-12311,8.889443,5.0,19.074886,17.001041,what is the difference between sex and love,her mantra &quot;no more bad dates&quot; is a ...,872.68384,0.0,0.0,0.0,0,"[8.889443285070843, 5.0, 19.074885717934308, 1...",0
3,1,clueweb12-1400tw-53-02910,7.796204,4.0,17.229402,15.395087,what is the difference between sex and love,"leave a comment in the past, i’ve had sex, i’v...",1970.16040,0.0,0.0,0.0,2,"[7.796203967373771, 4.0, 17.22940189701382, 15...",0
4,1,clueweb12-1801wb-11-09209,7.637733,4.0,14.292758,13.839917,what is the difference between sex and love,nor can we make love without god for he is the...,1178.09250,0.0,0.0,0.0,2,"[7.637732581490534, 4.0, 14.292757865862576, 1...",0


In [107]:
test_ds.head()

,qid,docno,score_pl2,score_tf,score_bm,score_dfic,query,text,baseline_scores,is_retrieved,ap_score,objs_score,label,features
0,40,clueweb12-1700tw-30-03487,6.597378,6.5,13.481996,13.584072,which operating system has better performance ...,windows 7 professional and ultimate users will...,2255.4893,1.0,0.0,2.0,0,"[6.597377968911245, 6.5, 13.481995561686743, 1..."
1,40,clueweb12-0300tw-05-10074,6.555987,6.5,13.607904,13.723079,which operating system has better performance ...,windows 8 is a coin with two very different si...,1917.8842,1.0,0.0,2.0,1,"[6.555986727741695, 6.5, 13.60790372983337, 13..."
2,40,clueweb12-1100tw-64-01206,5.987314,3.0,13.224987,11.186697,which operating system has better performance ...,it targeted even better performance for window...,1942.9227,1.0,1.0,2.0,2,"[5.987314476428924, 3.0, 13.224986571587602, 1..."
3,40,clueweb12-0700tw-41-20357,5.911963,8.0,11.903069,12.959875,which operating system has better performance ...,microsoft&#x27;s embedded windows operating sy...,2535.3645,1.0,0.0,2.0,0,"[5.911962693211354, 8.0, 11.903069461047062, 1..."
4,40,clueweb12-1700wb-29-15033,5.806860,8.0,12.168118,12.562240,which operating system has better performance ...,windows 8 improves on work on minwin which sta...,1950.5464,1.0,0.0,2.0,2,"[5.80685983863876, 8.0, 12.168117999890416, 12..."
